In [3]:
import google.oauth2.credentials
import google_auth_oauthlib.flow

# Use the client_secret.json file to identify the application requesting
# authorization. The client ID (from that file) and access scopes are required.
flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
    'client_secret.json',
    scopes=['https://www.googleapis.com/auth/youtube.force-ssl'])

# Indicate where the API server will redirect the user after the user completes
# the authorization flow. The redirect URI is required.
flow.redirect_uri = 'https://www.apriljharry.com'

# Generate URL for request to Google's OAuth 2.0 server.
# Use kwargs to set optional request parameters.
authorization_url, state = flow.authorization_url(
    # Enable offline access so that you can refresh an access token without
    # re-prompting the user for permission. Recommended for web server apps.
    access_type='offline',
    # Enable incremental authorization. Recommended as a best practice.
    include_granted_scopes='true')

In [4]:
authorization_url

'https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=780018333162-086aket7clh6ll4dh0eqieh1uegcv535.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fwww.apriljharry.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=sMul81nhWOKTXwEgiXSmJlZgS5rxZc&access_type=offline&include_granted_scopes=true'

In [5]:
state

'sMul81nhWOKTXwEgiXSmJlZgS5rxZc'

In [8]:
import flask
flask.redirect(authorization_url)

<Response 937 bytes [302 FOUND]>

In [9]:
state = flask.session['state']
flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
    'client_secret.json',
    scopes=['https://www.googleapis.com/auth/youtube.force-ssl'],
    state=state)
flow.redirect_uri = flask.url_for('oauth2callback', _external=True)

authorization_response = flask.request.url
flow.fetch_token(authorization_response=authorization_response)

# Store the credentials in the session.
# ACTION ITEM for developers:
#     Store user's access and refresh tokens in your data store if
#     incorporating this code into your real app.
credentials = flow.credentials
flask.session['credentials'] = {
    'token': credentials.token,
    'refresh_token': credentials.refresh_token,
    'token_uri': credentials.token_uri,
    'client_id': credentials.client_id,
    'client_secret': credentials.client_secret,
    'scopes': credentials.scopes}

RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request.  Consult the documentation on testing for
information about how to avoid this problem.

In [10]:
 # Authorize server-to-server interactions from Google Compute Engine.
 import httplib2
 from oauth2client.contrib import gce

 credentials = gce.AppAssertionCredentials(
   scope='https://www.googleapis.com/auth/devstorage.read_write')
 http = credentials.authorize(httplib2.Http())